# Import

In [ ]:
print('test test test')

In [ ]:
# Try and get keras plot to work
!pip install -q pydot
!pip install -q pydotplus
!apt-get install -q graphviz

import os
import numpy as np
import pandas as pd
from PIL import Image
import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_addons as tfa;
import scipy;
from collections import Counter
from datetime import datetime
import multiprocessing
from glob import glob
import warnings
import requests
import imageio
import IPython
import urllib
import zipfile
import pickle
import random
import shutil
import string
import math
import tqdm
import time
import gzip
import io
import os
import gc
import re
from matplotlib.colors import ListedColormap
import matplotlib.patches as patches
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import plotly.express as px
import seaborn as sns
import matplotlib; 
import plotly
import PIL
import cv2
import ast
try:
    import hpacellseg.cellsegmentator as cellsegmentator
    from hpacellseg.utils import label_cell,label_nuclei
    import pytorch_zoo
except:
    !pip install -q "/kaggle/input/hpacellsegmentatormaster/HPA-Cell-Segmentation-master"
    !pip install -q "/kaggle/input/hpapytorchzoozip/pytorch_zoo-master"
    import hpacellseg.cellsegmentator as cellsegmentator
    from hpacellseg.utils import label_cell,label_nuclei
    import pytorch_zoo

print("\n\nIMPORTS COMPLETE\n")

## 要复制到他们那里的

In [ ]:
# some functions used in segmentation visualization

def get_blended_image(images): 
    # get rgby images for sample

    # blend rgby images into single array
    blended_array = np.stack(images[:-1], 2)

    # Create PIL Image
    blended_image = Image.fromarray( np.uint8(blended_array) )
    return blended_image


def image_to_arrays(path):  
    # list of image filters. png -> np.arrays
    image_arrays = list()
    for image in path:
        array = np.asarray(Image.open(image))
        image_arrays.append(array)
    image_arrays = np.array(image_arrays)
    return image_arrays




In [ ]:
DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
colours = ['_red.png', '_blue.png', '_yellow.png', '_green.png']
TRAIN = '../input/hpa-single-cell-image-classification/train'
paths = [[os.path.join(TRAIN, train.iloc[idx,0])+ colour for colour in colours] for idx in range(len(train))]

titles = ['microtubules', 'nuclei', 'endoplasmic reticulum', 'protein of interest']
fig, axs = plt.subplots(2, 4, figsize =(16,8))
for entry in range(2):
    for channel in range(4):
        img = plt.imread(paths[entry][channel])
        axs[entry, channel].imshow(img)
        axs[entry, channel].set_xticks([])
        axs[entry, channel].set_yticks([])
        if entry == 0:
            axs[0, channel].set_title(titles[channel])


In [ ]:
NUC_MODEL = '/kaggle/input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth'
CELL_MODEL = '/kaggle/input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth'
segmentator = cellsegmentator.CellSegmentator(
    NUC_MODEL,
    CELL_MODEL,
    scale_factor=0.25,
    device="cuda",
    padding=False,
    multi_channel_model=True,
)
image_path = paths[10]   # shape:(21806, 4). 4 channels, rgby
arrays = image_to_arrays(image_path)  # shape: (4, 2048, 2048)
nuclei = arrays[1]
cell = arrays[:-1]
print('image shape:',arrays.shape)
print('nuclei shape:',nuclei.shape)
print('cell shape:',cell.shape)


# Nuclei segmentation
nuc_segmentations = segmentator.pred_nuclei([nuclei])

f, ax = plt.subplots(1, 2, figsize=(8,8))
ax[0].imshow(arrays[1])
ax[0].set_title('Original Nucleis', size=10)
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].imshow(nuc_segmentations[0])
ax[1].set_title('Segmented Nucleis', size=10)
plt.axis('off')
plt.show()

# Cell segmentation
inter_step = [[i] for i in image_path[:-1]]
cell_segmentations = segmentator.pred_cells(inter_step)

f, ax = plt.subplots(1, 2, figsize=(8,8))
ax[0].imshow(get_blended_image(arrays))
ax[0].set_title('Original Cells')
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].imshow(cell_segmentations[0])
ax[1].set_title('Segmented Cells')
plt.axis('off')
plt.show()

In [ ]:
# Nuclei mask
nuclei_mask = label_nuclei(nuc_segmentations[0])
# Cell masks
cell_nuclei_mask, cell_mask = label_cell(nuc_segmentations[0], cell_segmentations[0])
# Plotting
f, ax = plt.subplots(1, 3, figsize=(12,12))
ax[0].imshow(nuclei_mask)
ax[0].set_title('Nuclei Mask')
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].imshow(cell_nuclei_mask)
ax[1].set_title('Cell Nuclei Mask')
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[2].imshow(cell_mask)
ax[2].set_title('Cell Mask')
plt.axis('off')
plt.show()

In [ ]:
# segment cells

# Unique vector of cell_mask numbers
numbers = set(np.ravel(cell_mask))
numbers.remove(0)

plt.figure()
plt.title("Complete Cell Mask")
plt.imshow(cell_mask)
plt.axis('off')


# show several segments for visulazation

fig = plt.figure(figsize=(16,16))
index = 1
for number in numbers:
    number = number*4
    isolated_cell = np.where(cell_mask==number, cell_mask, 0)
    ax = fig.add_subplot(1, 5, index)
    ax.set_title("Segment"+str(number))
    ax.set_xticks([])
    ax.set_yticks([])
    plt.imshow(isolated_cell)
    index += 1
    if index == 6:
        break

## training dataset generalization

In [ ]:
# ROOT_DIR = "/kaggle/input"
DATA_DIR = "/kaggle/input/hpa-single-cell-image-classification"
# COMP_DIR = os.path.join(ROOT_DIR, "hpa-single-cell-image-classification")
# PKL_DIR = os.path.join(ROOT_DIR, "hpa-rule-based-single-cell-filtering")
# # paths of the training tiles 
# RED_TILE_DIR = os.path.join(ROOT_DIR, "human-protein-atlas-red-cell-tile-dataset")
# GREEN_TILE_DIR = os.path.join(ROOT_DIR, "human-protein-atlas-green-cell-tile-dataset")
# BLUE_TILE_DIR = os.path.join(ROOT_DIR, "human-protein-atlas-blue-cell-tile-dataset")
# YELLOW_TILE_DIR = os.path.join(ROOT_DIR, "human-protein-atlas-yellow-cell-tile-dataset")

In [ ]:
# Define the paths to the training and testing tfrecord and image folders respectively
TRAIN_IMG_DIR = os.path.join(DATA_DIR, "train")
TRAIN_TFREC_DIR = os.path.join(DATA_DIR, "train_tfrecords")
TEST_IMG_DIR = os.path.join(DATA_DIR, "test")
TEST_TFREC_DIR = os.path.join(DATA_DIR, "test_tfrecords")

In [ ]:
# LBL_NAMES = ["Nucleoplasm", "Nuclear Membrane", "Nucleoli", "Nucleoli Fibrillar Center", "Nuclear Speckles", "Nuclear Bodies", "Endoplasmic Reticulum", "Golgi Apparatus", "Intermediate Filaments", "Actin Filaments", "Microtubules", "Mitotic Spindle", "Centrosome", "Plasma Membrane", "Mitochondria", "Aggresome", "Cytosol", "Vesicles", "Negative"]
# INT_2_STR = {x:LBL_NAMES[x] for x in np.arange(19)}
# INT_2_STR_LOWER = {k:v.lower().replace(" ", "_") for k,v in INT_2_STR.items()}
# STR_2_INT_LOWER = {v:k for k,v in INT_2_STR_LOWER.items()}
# STR_2_INT = {v:k for k,v in INT_2_STR.items()}
# FIG_FONT = dict(family="Helvetica, Arial", size=14, color="#7f7f7f")
# LABEL_COLORS = [px.colors.label_rgb(px.colors.convert_to_RGB_255(x)) for x in sns.color_palette("Spectral", len(LBL_NAMES))]
# LABEL_COL_MAP = {str(i):x for i,x in enumerate(LABEL_COLORS)}

In [ ]:
# TILE_DIRS = [RED_TILE_DIR, GREEN_TILE_DIR, BLUE_TILE_DIR, YELLOW_TILE_DIR]
# # Define the paths to the training files for the tile dataset as a map from class index to list of paths
# RED_FILE_MAP, GREEN_FILE_MAP, BLUE_FILE_MAP, YELLOW_FILE_MAP = get_color_path_maps(TILE_DIRS, None)

In [ ]:
## helper function里的 有的话不用加

def load_image(img_id, img_dir):
    """ Load An Image Using ID and Directory Path - Composes 4 Individual Images """
    rgby = [
        np.asarray(Image.open(os.path.join(img_dir, img_id+f"_{c}.png")), np.uint8) \
        for c in ["red", "green", "blue", "yellow"]
    ]
    return np.stack(rgby, axis=-1)


def plot_rgb(arr, figsize=(12,12)):
    """ Plot 3 Channel Microscopy Image """
    plt.figure(figsize=figsize)
    plt.title(f"RGB Composite Image", fontweight="bold")
    plt.imshow(arr)
    plt.axis(False)
    plt.show()
    
    
def get_color_path_map(color_dir):
    c_p_map = {}
    for c in tqdm(os.listdir(color_dir), total=len(os.listdir(color_dir))):
        if c.endswith("256"):
            cls = c.split("_", 1)[1].rsplit("_",1)[0]
            c_dir = os.path.join(color_dir, c, "data", "train_tiles", cls)
            c_p_map[STR_2_INT_LOWER[cls]] = sorted([
                os.path.join(c_dir, f_name) \
                for f_name in  os.listdir(c_dir) \
                if f_name.endswith(".png")
            ])
    return c_p_map


def plot_ex(arr, figsize=(20,6), title=None, plot_merged=True, rgb_only=False):
    """ Plot 4 Channels Side by Side """
    if plot_merged and not rgb_only:
        n_images=5 
    elif plot_merged and rgb_only:
        n_images=4
    elif not plot_merged and rgb_only:
        n_images=4
    else:
        n_images=3
    plt.figure(figsize=figsize)
    if type(title) == str:
        plt.suptitle(title, fontsize=20, fontweight="bold")

    for i, c in enumerate(["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus", "Yellow – Endoplasmic Reticulum"]):
        if not rgb_only:
            ch_arr = np.zeros_like(arr[..., :-1])        
        else:
            ch_arr = np.zeros_like(arr)
        if c in ["Red Channel – Microtubles", "Green Channel – Protein of Interest", "Blue - Nucleus"]:
            ch_arr[..., i] = arr[..., i]
        else:
            if rgb_only:
                continue
            ch_arr[..., 0] = arr[..., i]
            ch_arr[..., 1] = arr[..., i]
        plt.subplot(1,n_images,i+1)
        plt.title(f"{c.title()}", fontweight="bold")
        plt.imshow(ch_arr)
        plt.axis(False)
        
    if plot_merged:
        plt.subplot(1,n_images,n_images)
        
        if rgb_only:
            plt.title(f"Merged RGB", fontweight="bold")
            plt.imshow(arr)
        else:
            plt.title(f"Merged RGBY into RGB", fontweight="bold")
            plt.imshow(convert_rgby_to_rgb(arr))
        plt.axis(False)
        
    plt.tight_layout(rect=[0, 0.2, 1, 0.97])
    plt.show()
    
    
# def flatten_list_of_lists(l_o_l, to_string=False):
#     if not to_string:
#         return [item for sublist in l_o_l for item in sublist]
#     else:
#         return [str(item) for sublist in l_o_l for item in sublist]

    
    
def grab_contours(cnts):
    # if the length the contours tuple returned by cv2.findContours
    # is '2' then we are using either OpenCV v2.4, v4-beta, or
    # v4-official
    if len(cnts) == 2:
        cnts = cnts[0]

    # if the length of the contours tuple is '3' then we are using
    # either OpenCV v3, v4-pre, or v4-alpha
    elif len(cnts) == 3:
        cnts = cnts[1]

    # otherwise OpenCV has changed their cv2.findContours return
    # signature yet again and I have no idea WTH is going on
    else:
        raise Exception(("Contours tuple must have length 2 or 3, "
            "otherwise OpenCV changed their cv2.findContours return "
            "signature yet again. Refer to OpenCV's documentation "
            "in that case"))

    # return the actual contours array
    return cnts

    
def convert_rgby_to_rgb(arr):
    """ Convert a 4 channel (RGBY) image to a 3 channel RGB image.
    
    Advice From Competition Host/User: lnhtrang

    For annotation (by experts) and for the model, I guess we agree that individual 
    channels with full range px values are better. 
    In annotation, we toggled the channels. 
    For visualization purpose only, you can try blending the channels. 
    For example, 
        - red = red + yellow
        - green = green + yellow/2
        - blue=blue.
        
    Args:
        arr (numpy array): The RGBY, 4 channel numpy array for a given image
    
    Returns:
        RGB Image
    """
    
    rgb_arr = np.zeros_like(arr[..., :-1])
    rgb_arr[..., 0] = arr[..., 0]
    rgb_arr[..., 1] = arr[..., 1]+arr[..., 3]/2
    rgb_arr[..., 2] = arr[..., 2]
    
    return rgb_arr

# def tif_gzip_to_png(tif_path):
#     """Function to convert .tif.gz to .png and put it in the same folder
    
#     Eg. for working in local work station
    
#     Args:
#         tif_path (str): Path to the tif zip file to be converted to png
        
#     Returns:
#         None; Zip will be unzipped to same directory as .tif zip exists    
#     """
    
#     png_path = pathlib.Path(tif_path.replace('.tif.gz','.png'))
#     tf = gzip.open(tif_path).read()
#     img = imageio.imread(tf, 'tiff')
#     imageio.imwrite(png_path, img)
    
    
# def download_and_convert_tifgzip_to_png(url, target_path):    
#     """Function to convert .tif.gz to .png and put it in the same folder
    
#     Args:
#         url (str): Path to the url containing the tifgzip file
#         target_path (str): Path to directory to unzip to
        
#     Returns:
#         None; Images are downloaded and unzipped    
#     """
    
#     r = requests.get(url)
#     f = io.BytesIO(r.content)
#     tf = gzip.open(f).read()
#     img = imageio.imread(tf, 'tiff')
#     imageio.imwrite(target_path, img)

    
# def get_new_data():
#     public_hpa_df = pd.read_csv('../input/publichpa/kaggle_2021.tsv',sep='\t',header=None)
#     public_hpa_df.columns = ['Image', 'Label']
#     colors = ['blue', 'red', 'green', 'yellow']
#     save_dir = os.path.join(os.getcwd(),'publichpa')
#     if not os.path.exists(save_dir):
#         os.makedirs(save_dir)

#     for i, row in public_hpa_df.iterrows():
#         try:
#             img = row.Image
#             for color in colors:
#                 img_url = f'{img}_{color}.tif.gz'
#                 save_path = os.path.join(save_dir,  f'{os.path.basename(img)}_{color}.png')
#                 download_and_convert_tifgzip_to_png(img_url, save_path)
#                 print(f'Downloaded {img_url} as {save_path}')    
#         except:
#             print(f'failed to download: {img}')
#     return save_dir

# def seg_demo_plot(mt, er, nu):
#     fig, ax = plt.subplots(3,3, figsize=(20,18))
#     for i in range(3):
#         microtubule = plt.imread(mt[i])    
#         endoplasmicrec = plt.imread(er[i])    
#         nuclei = plt.imread(nu[i])
#         mask = plt.imread(mt[i].replace('red','predictedmask'))
#         img = np.dstack((microtubule, endoplasmicrec, nuclei))
#         ax[0][i].imshow(img)
#         ax[1][i].imshow(mask)
#         ax[2][i].imshow(img)
#         ax[2][i].imshow(mask, alpha=0.5)

#         ax[0][i].axis('off')
#         ax[1][i].axis('off')
#         ax[2][i].axis('off')
#     plt.tight_layout()
#     plt.show()
    
    
# def multihot_melt(row):
#     """ Melt current column into 19 binary-1-hot columns """
#     return [1 if L in row["target_list"] else 0 for L in LBL_NAMES]



# Segmentation

In [ ]:
def encode_binary_mask(mask, mask_val=1):
    """Converts a binary mask into OID challenge encoding ascii text."""
    mask = np.where(mask==mask_val, 1, 0)
    
    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(f"encode_binary_mask expects a binary mask, received dtype == {mask.dtype}")

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(f"encode_binary_mask expects a 2d mask, received shape == {mask.shape}")

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str


def rle_encoding(img, mask_val=1):
    """
    Turns our masks into RLE encoding to easily store them
    and feed them into models later on
    https://en.wikipedia.org/wiki/Run-length_encoding
    
    Args:
        img (np.array): Segmentation array
        mask_val (int): Which value to use to create the RLE
        
    Returns:
        RLE string
    
    """
    dots = np.where(img.T.flatten() == mask_val)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
        
    return ' '.join([str(x) for x in run_lengths])


def rle_to_mask(rle_string, height, width):
    """ Convert RLE sttring into a binary mask 
    
    Args:
        rle_string (rle_string): Run length encoding containing 
            segmentation mask information
        height (int): Height of the original image the map comes from
        width (int): Width of the original image the map comes from
    
    Returns:
        Numpy array of the binary segmentation mask for a given cell
    """
    rows,cols = height,width
    rle_numbers = [int(num_string) for num_string in rle_string.split(' ')]
    rle_pairs = np.array(rle_numbers).reshape(-1,2)
    img = np.zeros(rows*cols,dtype=np.uint8)
    for index,length in rle_pairs:
        index -= 1
        img[index:index+length] = 255
    img = img.reshape(cols,rows)
    img = img.T
    return img


def create_segmentation_maps(list_of_image_lists, batch_size=4):
    """ Function to generate segmentation maps using CellSegmentator tool 
    
    Args:
        list_of_image_lists (list of lists):
            - [[micro-tubules(red)], [endoplasmic-reticulum(yellow)], [nucleus(blue)]]
        batch_size (int): Batch size to use in generating the segmentation masks
        
    Returns:
        List of lists containing RLEs for all the cells in all images
    """
    
    all_mask_rles = {}
    t1 = time.time()
    NUC_MODEL = '/kaggle/input/hpacellsegmentatormodelweights/dpn_unet_nuclei_v1.pth'
    CELL_MODEL = '/kaggle/input/hpacellsegmentatormodelweights/dpn_unet_cell_3ch_v1.pth'
    segmentator = cellsegmentator.CellSegmentator(NUC_MODEL,CELL_MODEL)
    print(f"CELL SEGMENTOR CLASS   {time.time()-t1:.2f}")
    print()
    
    for i in tqdm(range(0, len(list_of_image_lists[0]), batch_size), total=len(list_of_image_lists[0])//batch_size):
        
        # Get batch of images
        sub_images = [img_channel_list[i:i+batch_size] for img_channel_list in list_of_image_lists]
        # Do segmentation
        
        t1 = time.time()
        cell_segmentations = segmentator.pred_cells(sub_images)
        nuc_segmentations = segmentator.pred_nuclei(sub_images[2])
        print(f"MODEL PREDICT BATCH {i//batch_size+1}    {time.time()-t1:.2f}")

        # post-processing
        for j, path in enumerate(sub_images[0]):
            t1 = time.time()
            img_id = path.replace("_red.png", "").rsplit("/", 1)[1]
            nuc_mask, cell_mask = label_cell(nuc_segmentations[j], cell_segmentations[j])
            new_name = os.path.basename(path).replace('red','mask')
            all_mask_rles[img_id] = [rle_encoding(cell_mask, mask_val=k) for k in range(1, np.max(cell_mask)+1)]
            print(f"LABEL CELL BATCH {i+1}-{j+1}    {time.time()-t1:.2f}", )
        print()
    return all_mask_rles
    

def get_img_list(img_dir, return_ids=False, sub_n=1):
    """ Get image list in the format expected by the CellSegmentator tool """
    if sub_n is None:
        sub_n=len(glob(img_dir + '/' + f'*_red.png'))
    if return_ids:
        images = [sorted(glob(img_dir + '/' + f'*_{c}.png'))[:sub_n] for c in ["red", "yellow", "blue"]]
        return [x.replace("_red.png", "").rsplit("/", 1)[1] for x in images[0]], images
    else:
        return [sorted(glob(img_dir + '/' + f'*_{c}.png'))[:sub_n] for c in ["red", "yellow", "blue"]]
    
    
    
def get_contour_bbox(rle, width, height):
    """ Get bbox of contour as `xmin ymin xmax ymax`
    
    Args:
        rle (rle_string): Run length encoding containing 
            segmentation mask information
        height (int): Height of the original image the map comes from
        width (int): Width of the original image the map comes from
    
    Returns:
        Numpy array for a cell bounding box coordinates
    """
    
    cnts = grab_contours(
        cv2.findContours(
            rle_to_mask(rle, height, width).copy(), 
            cv2.RETR_EXTERNAL, 
            cv2.CHAIN_APPROX_SIMPLE
        ))
    x,y,w,h = cv2.boundingRect(cnts[0])
    return x,y,x+w,y+h


def pad_to_square(a):
    """ Pad an array `a` evenly until it is a square """
    if a.shape[1]>a.shape[0]: # pad height
        n_to_add = a.shape[1]-a.shape[0]
        top_pad = n_to_add//2
        bottom_pad = n_to_add-top_pad
        a = np.pad(a, [(top_pad, bottom_pad), (0, 0), (0, 0)], mode='constant')

    elif a.shape[0]>a.shape[1]: # pad width
        n_to_add = a.shape[0]-a.shape[1]
        left_pad = n_to_add//2
        right_pad = n_to_add-left_pad
        a = np.pad(a, [(0, 0), (left_pad, right_pad), (0, 0)], mode='constant')
    else:
        pass
    return a


def cut_out_cells(rgby, rles, resize_to=(224,224), square_off=True):
    """ Cut out the cells as padded square images 
    
    Args:
        rgby (np.array): 4 Channel image to be cut into tiles
        rles (list of RLE strings): List of run length encoding containing 
            segmentation mask information
        resize_to (tuple of ints, optional): The square dimension to resize the image to
        square_off (bool, optional): Whether to pad the image to a square or not
        
    Returns:
        list of square arrays representing squared off cell images
    """
    w,h = rgby.shape[:2]
    contour_bboxes = [get_contour_bbox(rle, w, h) for rle in rles]
    arrs = [rgby[bbox[1]:bbox[3], bbox[0]:bbox[2], :] for bbox in contour_bboxes]
    if square_off:
        arrs = [pad_to_square(arr) for arr in arrs]
        
    if resize_to is not None:
        arrs = [cv2.resize(pad_to_square(arr), resize_to, interpolation=cv2.INTER_CUBIC) for arr in arrs]
    
    return arrs

In [ ]:
## 新函数 记得加进 helper functions
def plot_rgby(arr, figsize=(20,6), title=None, plot_merged=False):
    """ Plot 4 Channels Side by Side """
    if plot_merged:
        n_images=5 
    else:
        n_images=4
    plt.figure(figsize=figsize)
    if type(title) == str:
        plt.suptitle(title)

    for i, c in enumerate(["Microtubles", "Protein of Interest", "Nucleus", "Endoplasmic Reticulum"]):
        ch_arr = np.zeros_like(arr[..., :-1])        
        if c in ["Microtubles", "Protein of Interest", "Nucleus"]:
            ch_arr[..., i] = arr[..., i]
        else:
            ch_arr[..., 0] = arr[..., i]
            ch_arr[..., 1] = arr[..., i]
        plt.subplot(1,n_images,i+1)
        plt.title(f"{c.title()}")
        plt.imshow(ch_arr)
        plt.axis(False)
        
    if plot_merged:
        plt.subplot(1,n_images,n_images)
        plt.title(f"Merged RGBY into RGB", fontweight="bold")
        plt.imshow(convert_rgby_to_rgb(arr))
        plt.axis(False)
        
    plt.tight_layout(rect=[0, 0.2, 1, 0.97])
    plt.show()

In [ ]:
## raw data --> segment -> box -> rle


testing_ids, testing_images = get_img_list(TRAIN_IMG_DIR, return_ids=True, sub_n=2)
testing_masks = create_segmentation_maps(testing_images, 2)

testing_id_1 = testing_ids[1]
testing_img_1 = load_image(testing_id_1, TRAIN_IMG_DIR)
testing_mask_1 = testing_masks[testing_id_1]
cell_boxes = cut_out_cells(testing_img_1, testing_mask_1)

print("\n ———— Original Image———— \n")
plot_ex(testing_img_1)

print("\n ———— The first 5 segmented cells from one images ———— \n")
    
for b in cell_boxes:
    plot_rgby(b, title=" ")

In [ ]:
for b in cell_boxes:
    plot_rgby(b, title="MY METHOD")

## for test 的路径

In [ ]:
output_DIR = "/kaggle/working/train_SegmentedCells"

if not os.path.isdir(output_DIR):
    os.makedirs(output_DIR,exist_ok=True)

red_DIR = os.path.join(output_DIR, "red")
blue_DIR = os.path.join(output_DIR, "blue")
yellow_DIR = os.path.join(output_DIR, "yellow")
green_DIR = os.path.join(output_DIR, "green")

DIRs = [red_DIR, blue_DIR,yellow_DIR,green_DIR]
for DIR in DIRs:
    if not os.path.isdir(DIR):
        os.makedirs(DIR,exist_ok=True)
        

In [ ]:
from PIL import Image
red = Image.fromarray(cell_boxes[0])
red.save("/kaggle/working/1","png")
    

In [ ]:

output_DIR = "/kaggle/working/train_SegmentedCells"

red_DIR = os.path.join(output_DIR, "red")
blue_DIR = os.path.join(output_DIR, "blue")
yellow_DIR = os.path.join(output_DIR, "yellow")
green_DIR = os.path.join(output_DIR, "green")

DIRs = [red_DIR, blue_DIR,yellow_DIR,green_DIR]
for DIR in DIRs:
    if not os.path.exists(DIR):
        os.makedirs(DIR)
        
for b in cell_boxes:
    plot_rgby(b, title="MY METHOD")

testing_ids, testing_images = get_img_list(TRAIN_IMG_DIR, return_ids=True, sub_n=2)
testing_masks = create_segmentation_maps(testing_images, 2)

testing_id_1 = testing_ids[1]
testing_img_1 = load_image(testing_id_1, TRAIN_IMG_DIR)
testing_mask_1 = testing_masks[testing_id_1]
cell_boxes = cut_out_cells(testing_img_1, testing_mask_1，resize_to=(224,224))

# 复制get_img_list过来 方便看它怎么运行的
def get_img_list(img_dir, return_ids=False, sub_n=1):
    """ Get image list in the format expected by the CellSegmentator tool """
    if sub_n is None:
        sub_n=len(glob(img_dir + '/' + f'*_red.png'))
    if return_ids:
        images = [sorted(glob(img_dir + '/' + f'*_{c}.png'))[:sub_n] for c in ["red", "yellow", "blue"]]
        return [x.replace("_red.png", "").rsplit("/", 1)[1] for x in images[0]], images
    else:
        return [sorted(glob(img_dir + '/' + f'*_{c}.png'))[:sub_n] for c in ["red", "yellow", "blue"]]

In [ ]:
# 没什么用 用来看数据结构的
print(np.array(testing_ids).shape)
print(np.array(testing_images).shape)
print(TRAIN_IMG_DIR)
sub_n=len(glob(TRAIN_IMG_DIR + '/' + f'*_red.png'))
print(sub_n)  # 21806 red pictures
print('id: ', testing_id_1)

In [ ]:

train_IDs, train_images_ryb = get_img_list(TRAIN_IMG_DIR, return_ids=True, sub_n=2)
train_masks = create_segmentation_maps(train_images_ryb, 2)
train_img_rgby = load_image(train_IDs, TRAIN_IMG_DIR)

cell_boxes = cut_out_cells(train_img_rgby, train_masks，resize_to=(224,224))